### Libraries

In [ ]:
import pandas as pd
import gdown  # to download data from google drive
import gzip   # to decompress downloaded data
import shutil
from scipy import stats
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

### Functions to download raw data

In [ ]:
def download_data(folder_id):
    '''
    Downloads all content in a given google drive folder.

    Args:
    folder_id (str): google drive folder id which the content is to be downloaded from.
    '''
    print("downloading")
    url = f"https://drive.google.com/drive/folders/{folder_id}"
    # gdown URL formate
    gdown.download_folder(url=url, output="./downloaded_folder", quiet=False, use_cookies=False)

In [ ]:
def decompress_gz(file_path, output_file_name):
    '''
    Decompresses given gzip file.

    Args:
    file_path (str): path of gzip file to be decompressed.
    output_file_name (str): name to be save of file after decompression.
    '''
    print("decompressing")
    with gzip.open(file_path, 'rb') as f_in:
        with open(output_file_name, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [ ]:
def raw_data_downloading(file_path, downloaded_directory, decompressed_file_name):
    '''
    Downloads raw dataset.

    Args:
    file_path (str): path of dataset gzip file on google drive.
    downloaded_directory (str): path of dataset gzip file after download.
    decompressed_file_name (str): name to be save of file after decompression.
    '''
    download_data(file_path)
    decompress_gz(downloaded_directory, decompressed_file_name)

### Cleaning

In [ ]:
def reading_data_into_dataframe(file_name):
    '''
    Saves raw dataset as a dataframe.

    Args:
    file_path (str): Dataset directory.

    Returns:
    dataframe: created dataframe
    '''
    print("reading")
    accepted_df = pd.read_csv(file_name, low_memory=False)
    return accepted_df

In [ ]:
def high_null_drop(accepted_df, important_cols):
    '''
    Drops unimportant columns with high null count.

    Args:
    accepted_df (dataframe): Dataset dataframe.
    important_cols (list(str)): Columns of importance with high null count to avoid dropping.
    '''
    print("col high null drop")
    number_of_entries = accepted_df.shape[0]
    for col in accepted_df.columns.tolist():
        if col in important_cols:
            continue
        tot_col_nulls = accepted_df[col].isna().sum()
        if tot_col_nulls >= (0.2 * number_of_entries):
            accepted_df.drop(col, axis=1, inplace=True)

In [ ]:
def fix_null(accepted_df, col_2b_fixed, col_used_to_fix):
    '''
    Adjusts null values of a given column to 0 based on another related column.

    Args:
    accepted_df (dataframe): Dataset dataframe.
    col_2b_fixed (str): Name of column that needs adjustement.
    col_used_to_fix (str): Name of column used in fixing errored column.
    '''
    print("fixing delinq")
    wrong_col_index = accepted_df.columns.get_loc(col_2b_fixed)
    fixer_col_index = accepted_df.columns.get_loc(col_used_to_fix)
    for i in range (accepted_df.shape[0]):
        if accepted_df.iat[i, fixer_col_index] == 0 and accepted_df.iat[i, wrong_col_index]:
            accepted_df.iat[i, wrong_col_index] = 0

In [ ]:
def drop_excessive_nulls(accepted_df):
    '''
    Drops all rows with null values.

    Args:
    accepted_df (dataframe): Dataset dataframe.
    '''
    print("nulled rows drop")
    for col in accepted_df.columns.tolist():
        accepted_df = accepted_df[accepted_df[col].notna()]
    return accepted_df

In [ ]:
def drop_unecessary_cols(accepted_df, columns):
    '''
    Drops columns unecessary for prediction.

    Args:
    accepted_df (dataframe): Dataset dataframe.
    columns (list(str)): Columns' names that are unecessary.
    '''
    print("unnecessary cols drop")
    for col in columns:
        if col in accepted_df.columns.tolist():
            accepted_df.drop(col, axis=1, inplace=True)

In [ ]:
def encoding_target_var(df):
    '''
    Encodes the target variable categories into "1" for good credit & "0" for bad credit.

    Args:
    df (dataframe): Dataset dataframe.
    '''
    print("encoding trgt var")
    df['loan_status'] = df['loan_status'].replace('Current', "1")
    df['loan_status'] = df['loan_status'].replace('Fully Paid', "1")
    df['loan_status'] = df['loan_status'].replace('Charged Off', "0")
    df['loan_status'] = df['loan_status'].replace('Late (31-120 days)', "0")
    df['loan_status'] = df['loan_status'].replace('In Grace Period', "1")
    df['loan_status'] = df['loan_status'].replace('Late (16-30 days)', "1")
    df['loan_status'] = df['loan_status'].replace('Default', "0")

In [ ]:
def sampling(df):
    '''
    Randomly chooses rows with target "1" to match number of target "0" instances.

    Args:
    df (dataframe): Dataset dataframe.

    Returns:
    dataframe: Adjusted dataframe.
    '''
    print("sampling")
    subset_good = df[df['loan_status'] == "1"].sample(n=df[df['loan_status'] == "0"].shape[0],
                                                      random_state=737)
    subset_bad = df[df['loan_status'] == "0"]
    df = pd.concat([subset_good, subset_bad])
    df = df.sample(frac = 1)
    df.reset_index(inplace=True)
    return df

In [ ]:
def outlier_removal(df):
    '''
    Determines outliers using z-score then drop them.

    Args:
    df (dataframe): Dataset dataframe.

    Returns:
    dataframe: Adjusted dataframe.
    '''
    print("removing outliers")
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col != 'loan_status']

    z_scores = np.abs(stats.zscore(df[numeric_cols]))

    threshold_z = 3  # thershold to determine outlier

    outlier_rows = (z_scores > threshold_z).any(axis=1)

    df = df[~outlier_rows]  # removal of outliers

    df.reset_index(drop=True, inplace=True)

    return df

In [ ]:
def standardizing(df):
    '''
    Standardizing all numeric columns of dataframe.

    Args:
    df (dataframe): Dataset dataframe.
    '''
    print("standardizing")
    scaler = StandardScaler()
    for col in df.columns:
        if df[col].dtype != 'O' and col != "loan_status":
            col_array = np.array(df[col])
            col_array = col_array.reshape(-1, 1)
            scaler.fit(col_array)
            df[col] = scaler.transform(col_array)

In [ ]:
def label_encoding(df):
    '''
    Label encodes all object columns of dataframe.

    Args:
    df (dataframe): Dataset dataframe.
    '''
    print("label encoding")
    le = LabelEncoder()
    for col in df.columns:
        if df[col].dtype == 'O':
            le.fit(df[col])
            df[col] = le.transform(df[col])

In [ ]:
def data_cleaning(csv_file):
    '''
    Performs all data cleaning.

    Args:
    csv_file (str): CSV file of dataset

    Returns:
    dataframe: Cleaned dataframe.
    '''
    df = reading_data_into_dataframe(csv_file)
    high_null_drop(df, ["mths_since_last_delinq", "all_util"])
    fix_null(df, "mths_since_last_delinq", "delinq_amnt")
    df = drop_excessive_nulls(df)
    drop_unecessary_cols(df, ["acceptD", "application_type", "creditPullD", "desc",
                   "emp_title", "expD", "id", "listD", "mthsSinceMostRecentInq",
                   "reviewStatusD", "title", "url", "zip_code", "sec_app_inq_last_6mths"])
    encoding_target_var(df)
    df = sampling(df)
    df = outlier_removal(df)
    standardizing(df)
    label_encoding(df)
    df.drop("index", axis=1, inplace=True)
    return df

### Main

In [ ]:
raw_data_downloading("14ZG8utOf0Ry76w_T9rqrk5kdhhPIK84P", '/kaggle/working/downloaded_folder/accepted_2007_to_2018Q4.csv.gz',
                "accepted.csv")
df = data_cleaning("accepted.csv")
print(df.shape)
df.head()

reading
col high null drop
fixing delinq
nulled rows drop
unnecessary cols drop
encoding trgt var
sampling
removing outliers


/tmp/ipykernel_31/554659916.py:15: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = np.abs(stats.zscore(df[numeric_cols]))
/tmp/ipykernel_31/554659916.py:19: RuntimeWarning: invalid value encountered in greater
  outlier_rows = (z_scores > threshold_z).any(axis=1)


standardizing
label encoding
(119286, 89)


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,2.705294,2.705294,2.706406,0,-0.800165,3.010788,1,6,1,2,...,1.691136,-0.42679,-0.151487,-0.413588,0.596422,0.408736,0.616524,0,1,0
1,2.831965,2.831965,2.833102,1,0.936464,2.194618,3,17,1,3,...,-0.559035,-0.42679,-0.151487,0.089148,2.015826,0.481396,2.387143,0,0,0
2,-0.461473,-0.461473,-0.461004,0,-1.252451,-0.430270,0,2,4,3,...,-1.121577,-0.42679,-0.151487,-0.929017,-1.011710,-0.238594,-0.922073,0,0,0
3,2.071941,2.071941,2.072924,0,0.064333,2.605715,2,13,1,1,...,-0.066810,-0.42679,-0.151487,0.193963,-0.319778,0.791850,-0.686441,0,0,0
4,-0.461473,-0.461473,-0.461004,1,0.547154,-0.695067,3,15,1,3,...,0.754502,-0.42679,-0.151487,-0.648830,-0.004733,-0.007405,0.040135,0,0,0


### Downloading cleaned data

In [ ]:
df.to_csv('df_cleaned.csv', index=False)